In [1]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 데이터 파일들이 노트북 파일과 같은 디렉토리에 있다고 가정합니다.
df1 = pd.read_csv('homework_3.1.csv')
df2a = pd.read_csv('homework_3.2.a.csv')
df2b = pd.read_csv('homework_3.2.b.csv')

print("데이터 로딩 완료.")
df1.head()

데이터 로딩 완료.


,Unnamed: 0,time,value1,value2,value3
0,0,0,1.764052,1.883151,-0.369182
1,1,1,0.420157,-1.327759,-0.219379
2,2,2,1.018738,-1.230485,1.139660
3,3,3,2.300893,1.029397,0.715264
4,4,4,1.947558,-1.093123,0.720132


For questions 1 and 2: 

Given a dataset with time series data containing an event, use a linear regression to test whether there was a discontinuity in the data at the event. Consider the possibility, first, of a discontinuity only in the value of the variable but not the derivative. Then consider that there may be a discontinuity in the first derivative (the slope).  
Use the file homework_3.1.csv. 

In [2]:
# time = 50을 기준으로 event 변수 생성 (0 또는 1)
df1['event'] = (df1['time'] >= 50).astype(int)

# time 변수를 불연속 지점을 중심으로 조정
df1['time_centered'] = df1['time'] - 50

# event와 조정된 time 변수 간의 상호작용 항 생성
df1['time_event_interaction'] = df1['time_centered'] * df1['event']

df1.head()

,Unnamed: 0,time,value1,value2,value3,event,time_centered,time_event_interaction
0,0,0,1.764052,1.883151,-0.369182,0,-50,0
1,1,1,0.420157,-1.327759,-0.219379,0,-49,0
2,2,2,1.018738,-1.230485,1.139660,0,-48,0
3,3,3,2.300893,1.029397,0.715264,0,-47,0
4,4,4,1.947558,-1.093123,0.720132,0,-46,0


In [ ]:
# 값의 불연속성(jump at the cutoff)을 확인하기 위한 모델
model1_val = smf.ols('value1 ~ time_centered + event', data=df1).fit()
model2_val = smf.ols('value2 ~ time_centered + event', data=df1).fit()
model3_val = smf.ols('value3 ~ time_centered + event', data=df1).fit()

print("--- Question 1: Discontinuity in Value (event 계수) ---")
print(f"Value1 event coefficient: {model1_val.params['event']:.4f}")
print(f"Value2 event coefficient: {model2_val.params['event']:.4f}")
print(f"Value3 event coefficient: {model3_val.params['event']:.4f}")


--- Question 1: Discontinuity in Value (event 계수) ---
Value1 event coefficient: 0.8508
Value2 event coefficient: 0.6827
Value3 event coefficient: 1.7673


In [6]:
# 기울기의 불연속성(change in slope)을 확인하기 위한 모델
model1_der = smf.ols('value1 ~ time_centered + event + time_event_interaction', data=df1).fit()
model2_der = smf.ols('value2 ~ time_centered + event + time_event_interaction', data=df1).fit()
model3_der = smf.ols('value3 ~ time_centered + event + time_event_interaction', data=df1).fit()

print("\n--- Question 2: Discontinuity in Derivative (interaction 계수) ---")
print(f"Value1 interaction coefficient: {model1_der.params['time_event_interaction']:.4f}")
print(f"Value2 interaction coefficient: {model2_der.params['time_event_interaction']:.4f}")
print(f"Value3 interaction coefficient: {model3_der.params['time_event_interaction']:.4f}")



--- Question 2: Discontinuity in Derivative (interaction 계수) ---
Value1 interaction coefficient: 0.1053
Value2 interaction coefficient: 0.0369
Value3 interaction coefficient: 0.0507


For questions 3 to 5:  

Given a dataset with treatment and control data having “before” and “after” parts, apply a differences-in-differences regression.  
Use homework_3.2.a.csv and homework_3.2.b.csv. 

In [9]:
# ============== Q3–Q5: DiD (3.2.a = Group 1, 3.2.b = Group 2) ==============
def run_did(df, y, treat, post):
    d = df.copy()
    d['treat'] = d[treat].astype(int)
    d['post']  = d[post].astype(int)
    d['y'] = d[y]
    d['treat_post'] = d['treat'] * d['post']
    m = smf.ols('y ~ treat + post + treat_post', data=d).fit(cov_type='HC1')
    # means cross-check
    g = d.groupby(['treat','post'])['y'].mean().unstack()
    did_means = (g.loc[1,1]-g.loc[1,0]) - (g.loc[0,1]-g.loc[0,0])
    return m, did_means, g

# Group 1 (3.2.a): outcome1, group1, time1
m1, did1, means1 = run_did(df2a, 'outcome1', 'group1', 'time1')
# Group 2 (3.2.b): outcome2, group2, time2
m2, did2, means2 = run_did(df2b, 'outcome2', 'group2', 'time2')

coef1, se1, p1 = m1.params['treat_post'], m1.bse['treat_post'], m1.pvalues['treat_post']
coef2, se2, p2 = m2.params['treat_post'], m2.bse['treat_post'], m2.pvalues['treat_post']

print("\n--- Q3–Q5: Differences-in-Differences 결과 ---")
print("Group 1 (3.2.a):")
print(f"  DiD coef (treat_post) = {coef1:.4f}  SE={se1:.4f}  p={p1:.3g}  | means-check = {did1:.4f}")
print("  Means table:\n", means1, "\n")

print("Group 2 (3.2.b):")
print(f"  DiD coef (treat_post) = {coef2:.4f}  SE={se2:.4f}  p={p2:.3g}  | means-check = {did2:.4f}")
print("  Means table:\n", means2)

# ===== MCQ 매핑 =====
q3_answer = "Group 1" if abs(coef1) > abs(coef2) else "Group 2"
q4_answer = "Group 1" if p1 < p2 else "Group 2"

def closest(val, options):
    return min(options, key=lambda x: abs(val - x))

opt_vals = [0.5831, 4.209, 1.248]  # Q5 보기
q5_closest = closest(coef2, opt_vals)

print("\n--- MCQ Picks ---")
print(f"Q3 (더 큰 처리효과): {q3_answer}")
print(f"Q4 (더 유의미함, 더 작은 p): {q4_answer}")
print(f"Q5 (Group 2 처리효과와 가장 가까움): {q5_closest}")


--- Q3–Q5: Differences-in-Differences 결과 ---
Group 1 (3.2.a):
  DiD coef (treat_post) = 0.6858  SE=0.0625  p=5.25e-28  | means-check = 0.6858
  Means table:
 post          0         1
treat                    
0     -0.025849  1.401364
1      1.960429  4.073489 

Group 2 (3.2.b):
  DiD coef (treat_post) = 1.3499  SE=0.1470  p=4.28e-20  | means-check = 1.3499
  Means table:
 post          0         1
treat                    
0      0.102107  1.367621
1      1.949826  4.565198

--- MCQ Picks ---
Q3 (더 큰 처리효과): Group 2
Q4 (더 유의미함, 더 작은 p): Group 1
Q5 (Group 2 처리효과와 가장 가까움): 1.248
